In [2]:
import argparse
import os
import shutil
from pathlib import Path

import numpy as np
import SimpleITK as sitk
from nnunetv2.paths import nnUNet_raw
from tqdm import tqdm

from nnunetv2.dataset_conversion.generate_dataset_json import \
    generate_dataset_json

In [3]:
MODALITY_ENCODINGS = {
    'T2': 0,
}

In [4]:
base_path = Path(os.getcwd()).parent

In [5]:
all_data_folder = base_path / "lab" / "Npz_files"
task_name = "MR_Crohns_Proxy_axial"
task_id = 1

In [29]:
npz_imgs_folder = all_data_folder / task_name / "data"
npz_weak_masks_folder = all_data_folder / task_name / "masks"
output_dataset_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}"
output_dataset_folder.mkdir(exist_ok=True)

# Train set
images_folder = output_dataset_folder / "imagesTr"
labels_folder = output_dataset_folder / "labelsTr"
images_folder.mkdir(exist_ok=True)
labels_folder.mkdir(exist_ok=True)

npz_imgs_paths = sorted(npz_imgs_folder.glob("*.npz"))
npz_weak_masks_paths = sorted(npz_weak_masks_folder.glob("*.npz"))

for i, (img_file, gt_file) in tqdm(
    enumerate(zip(npz_imgs_paths, npz_weak_masks_paths)),
    total=len(npz_imgs_paths),
):
    img_data = np.load(img_file)
    image_volume = img_data["imgs"]
    gt_data = np.load(gt_file)
    weak_mask_volume = gt_data["gts"]

    # convert the 3-channel image to a single channel image
    image_volume = image_volume[:, :, :, 0]

    # Save the image volume
    saved_image = sitk.GetImageFromArray(image_volume)
    saved_image.SetSpacing((1, 1, 1))
    saved_image.SetOrigin((0, 0, 0))
    saved_image.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_image,
        str(images_folder / f"{task_name}_Training_{i+1:03d}_0000.nii.gz"),
    )


    # Save the weak mask volume
    saved_weak_mask = sitk.GetImageFromArray(weak_mask_volume)
    saved_weak_mask.SetSpacing((1, 1, 1))
    saved_weak_mask.SetOrigin((0, 0, 0))
    saved_weak_mask.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_weak_mask,
        str(labels_folder / f"{task_name}_Training_{i+1:03d}.nii.gz"),
    )

100%|██████████| 97/97 [00:02<00:00, 39.69it/s]


In [30]:
generate_dataset_json(
    output_folder=str(output_dataset_folder),
    channel_names={
        encoding: modality for modality, encoding in MODALITY_ENCODINGS.items()
    },
    labels={
        "background": 0,
        "terminal ileum": 1,
    },
    num_training_cases=len(npz_imgs_paths),
    file_ending=".nii.gz",
    regions_class_order=(1,),
    license="see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863",
    reference="see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863",
    dataset_release_date="1.0",
)

In [31]:
commands = [
    "nnUNetv2_plan_and_preprocess",
    "-d",
    str(task_id),
    "--verify_dataset_integrity",
]

print(" ".join(commands))

nnUNetv2_plan_and_preprocess -d 1 --verify_dataset_integrity


In [7]:
import subprocess

trainer = "nnUNetTrainer_10epochs"
for fold in range(1, 5):
    commands = [
        "nnUNetv2_train",
        str(task_id),
        "2d",
        str(fold),
        "-device",
        "cuda",
        '--npz',
        '-tr',
        trainer,
    ]
    print(" ".join(commands))
    subprocess.run(commands)

nnUNetv2_train 1 2d 1 -device cuda --npz -tr nnUNetTrainer_10epochs
nnUNetv2_train 1 2d 2 -device cuda --npz -tr nnUNetTrainer_10epochs
nnUNetv2_train 1 2d 3 -device cuda --npz -tr nnUNetTrainer_10epochs
nnUNetv2_train 1 2d 4 -device cuda --npz -tr nnUNetTrainer_10epochs


In [7]:
import subprocess


trainer = "nnUNetTrainer_10epochs"
commands = [
    "nnUNetv2_find_best_configuration",
    str(task_id),
    "-c",
    "2d",
    "-tr",
    trainer,
]
print(" ".join(commands))
subprocess.run(commands)

nnUNetv2_find_best_configuration 1 -c 2d -tr nnUNetTrainer_10epochs


CompletedProcess(args=['nnUNetv2_find_best_configuration', '1', '-c', '2d', '-tr', 'nnUNetTrainer_10epochs'], returncode=0)

In [6]:
all_data_folder = base_path / "lab" / "Npz_files"
task_name = "MR_Crohns_axial"
task_id = 2

In [11]:
npz_data_folder = all_data_folder / task_name / "train"

output_dataset_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}"
output_dataset_folder.mkdir(exist_ok=True)

# Train set
images_folder = output_dataset_folder / "imagesTr"
labels_folder = output_dataset_folder / "labelsTr"
images_folder.mkdir(exist_ok=True)
labels_folder.mkdir(exist_ok=True)

npz_data_paths = sorted(npz_data_folder.glob("*.npz"))

for i, npz_file in tqdm(enumerate(npz_data_paths), total=len(npz_data_paths)):
    npz_data = np.load(npz_file)
    image_volume = npz_data["imgs"]
    gt_volume = npz_data["gts"]

    # convert the 3-channel image to a single channel image
    if image_volume.shape[-1] == 3:
        image_volume = image_volume[:, :, :, 0]

    # Save the image volume
    saved_image = sitk.GetImageFromArray(image_volume)
    saved_image.SetSpacing((1, 1, 1))
    saved_image.SetOrigin((0, 0, 0))
    saved_image.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_image,
        str(images_folder / f"{task_name}_Training_{i+1:03d}_0000.nii.gz"),
    )


    # Save the gt volume
    saved_gt = sitk.GetImageFromArray(gt_volume)
    saved_gt.SetSpacing((1, 1, 1))
    saved_gt.SetOrigin((0, 0, 0))
    saved_gt.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_gt,
        str(labels_folder / f"{task_name}_Training_{i+1:03d}.nii.gz"),
    )

100%|██████████| 24/24 [00:00<00:00, 33.86it/s]


In [13]:
generate_dataset_json(
    output_folder=str(output_dataset_folder),
    channel_names={
        encoding: modality for modality, encoding in MODALITY_ENCODINGS.items()
    },
    labels={
        "background": 0,
        "terminal ileum": 1,
    },
    num_training_cases=len(npz_data_paths),
    file_ending=".nii.gz",
    regions_class_order=(1,),
    license="see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863",
    reference="see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863",
    dataset_release_date="1.0",
)

In [14]:
commands = [
    "nnUNetv2_plan_and_preprocess",
    "-d",
    str(task_id),
    "--verify_dataset_integrity",
]

print(" ".join(commands))

nnUNetv2_plan_and_preprocess -d 2 --verify_dataset_integrity


In [15]:
from nnunetv2.paths import nnUNet_results
for fold in range(5):
    commands = [
        'nnUNetv2_train',
        str(task_id),
        '2d',
        str(fold),
        '-pretrained_weights',
        str(Path(nnUNet_results) / "Dataset001_MR_Crohns_Proxy_axial" / "nnUNetTrainer_200epochs__nnUNetPlans__2d" / "fold_all" / "checkpoint_best.pth"),
        '-tr',
        'nnUNetTrainer_50epochs',
    ]
    print(" ".join(commands))
    subprocess.run(commands)

nnUNetv2_train 2 2d 0 -pretrained_weights D:\Frank\Imperial-FYP\software_archive\data\nnUNet_results\Dataset001_MR_Crohns_Proxy_axial\nnUNetTrainer_200epochs__nnUNetPlans__2d\fold_all\checkpoint_best.pth -tr nnUNetTrainer_50epochs
nnUNetv2_train 2 2d 1 -pretrained_weights D:\Frank\Imperial-FYP\software_archive\data\nnUNet_results\Dataset001_MR_Crohns_Proxy_axial\nnUNetTrainer_200epochs__nnUNetPlans__2d\fold_all\checkpoint_best.pth -tr nnUNetTrainer_50epochs
nnUNetv2_train 2 2d 2 -pretrained_weights D:\Frank\Imperial-FYP\software_archive\data\nnUNet_results\Dataset001_MR_Crohns_Proxy_axial\nnUNetTrainer_200epochs__nnUNetPlans__2d\fold_all\checkpoint_best.pth -tr nnUNetTrainer_50epochs
nnUNetv2_train 2 2d 3 -pretrained_weights D:\Frank\Imperial-FYP\software_archive\data\nnUNet_results\Dataset001_MR_Crohns_Proxy_axial\nnUNetTrainer_200epochs__nnUNetPlans__2d\fold_all\checkpoint_best.pth -tr nnUNetTrainer_50epochs
nnUNetv2_train 2 2d 4 -pretrained_weights D:\Frank\Imperial-FYP\software_arc

In [16]:
npz_data_folder = all_data_folder / task_name / "val"

output_dataset_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}"
output_dataset_folder.mkdir(exist_ok=True)

# Train set
images_folder = output_dataset_folder / "imagesTs"
labels_folder = output_dataset_folder / "labelsTs"
images_folder.mkdir(exist_ok=True)
labels_folder.mkdir(exist_ok=True)

npz_data_paths = sorted(npz_data_folder.glob("*.npz"))

for i, npz_file in tqdm(enumerate(npz_data_paths), total=len(npz_data_paths)):
    npz_data = np.load(npz_file)
    image_volume = npz_data["imgs"]
    gt_volume = npz_data["gts"]

    # convert the 3-channel image to a single channel image
    if image_volume.shape[-1] == 3:
        image_volume = image_volume[:, :, :, 0]

    # Save the image volume
    saved_image = sitk.GetImageFromArray(image_volume)
    saved_image.SetSpacing((1, 1, 1))
    saved_image.SetOrigin((0, 0, 0))
    saved_image.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_image,
        str(images_folder / f"{task_name}_Testing_{i+1:03d}_0000.nii.gz"),
    )


    # Save the gt volume
    saved_gt = sitk.GetImageFromArray(gt_volume)
    saved_gt.SetSpacing((1, 1, 1))
    saved_gt.SetOrigin((0, 0, 0))
    saved_gt.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_gt,
        str(labels_folder / f"{task_name}_Testing_{i+1:03d}.nii.gz"),
    )

100%|██████████| 6/6 [00:00<00:00, 35.65it/s]


In [18]:
all_data_folder = base_path / "lab" / "Npz_files"
task_name = "MR_Crohns_axial"
task_id = 2
input_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}" / "imagesTs"
output_folder = base_path / "inference" / f"Dataset{task_id:03d}_{task_name}"
output_folder.mkdir(exist_ok=True)
print(f"nnUNetv2_predict -d Dataset{task_id:03d}_{task_name} -i {str(input_folder)} -o {str(output_folder)} -f  0 1 2 3 4 -tr nnUNetTrainer_50epochs -c 2d -p nnUNetPlans")

nnUNetv2_predict -d Dataset002_MR_Crohns_axial -i D:\Frank\Imperial-FYP\software_archive\data\nnUNet_raw\Dataset002_MR_Crohns_axial\imagesTs -o d:\Frank\Imperial-FYP\software_archive\inference\Dataset002_MR_Crohns_axial -f  0 1 2 3 4 -tr nnUNetTrainer_50epochs -c 2d -p nnUNetPlans


In [13]:
def compute_dice_coefficient(mask_gt, mask_pred):
    """Compute soerensen-dice coefficient.

    compute the soerensen-dice coefficient between the ground truth mask `mask_gt`
    and the predicted mask `mask_pred`.

    Args:
      mask_gt: 3-dim Numpy array of type bool. The ground truth mask.
      mask_pred: 3-dim Numpy array of type bool. The predicted mask.

    Returns:
      the dice coeffcient as float. If both masks are empty, the result is NaN
    """
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN
    volume_intersect = (mask_gt & mask_pred).sum()
    return 2 * volume_intersect / volume_sum

In [21]:
all_data_folder = base_path / "lab" / "Npz_files"
task_name = "MR_Crohns_axial"
task_id = 2
input_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}" / "imagesTs"
output_folder = base_path / "inference" / f"Dataset{task_id:03d}_{task_name}"
mask_prediction_paths = sorted(output_folder.glob("*.nii.gz"))
mask_gt_paths = sorted((Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}" / "labelsTs").glob("*.nii.gz"))
mask_prediction_paths

[WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset002_MR_Crohns_axial/MR_Crohns_axial_Testing_001.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset002_MR_Crohns_axial/MR_Crohns_axial_Testing_002.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset002_MR_Crohns_axial/MR_Crohns_axial_Testing_003.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset002_MR_Crohns_axial/MR_Crohns_axial_Testing_004.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset002_MR_Crohns_axial/MR_Crohns_axial_Testing_005.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset002_MR_Crohns_axial/MR_Crohns_axial_Testing_006.nii.gz')]

In [22]:
mask_gt_paths

[WindowsPath('D:/Frank/Imperial-FYP/software_archive/data/nnUNet_raw/Dataset002_MR_Crohns_axial/labelsTs/MR_Crohns_axial_Testing_001.nii.gz'),
 WindowsPath('D:/Frank/Imperial-FYP/software_archive/data/nnUNet_raw/Dataset002_MR_Crohns_axial/labelsTs/MR_Crohns_axial_Testing_002.nii.gz'),
 WindowsPath('D:/Frank/Imperial-FYP/software_archive/data/nnUNet_raw/Dataset002_MR_Crohns_axial/labelsTs/MR_Crohns_axial_Testing_003.nii.gz'),
 WindowsPath('D:/Frank/Imperial-FYP/software_archive/data/nnUNet_raw/Dataset002_MR_Crohns_axial/labelsTs/MR_Crohns_axial_Testing_004.nii.gz'),
 WindowsPath('D:/Frank/Imperial-FYP/software_archive/data/nnUNet_raw/Dataset002_MR_Crohns_axial/labelsTs/MR_Crohns_axial_Testing_005.nii.gz'),
 WindowsPath('D:/Frank/Imperial-FYP/software_archive/data/nnUNet_raw/Dataset002_MR_Crohns_axial/labelsTs/MR_Crohns_axial_Testing_006.nii.gz')]

In [23]:
all_data_folder = base_path / "lab" / "Npz_files"
task_name = "MR_Crohns_axial"
task_id = 2
input_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}" / "imagesTs"
output_folder = base_path / "inference" / f"Dataset{task_id:03d}_{task_name}"
mask_prediction_paths = sorted(output_folder.glob("*.nii.gz"))
mask_gt_paths = sorted((Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}" / "labelsTs").glob("*.nii.gz"))
dscs = []
for mask_pred_path, mask_gt_path in zip(mask_prediction_paths, mask_gt_paths):
    mask_pred = sitk.GetArrayFromImage(sitk.ReadImage(str(mask_pred_path)))
    mask_gt = sitk.GetArrayFromImage(sitk.ReadImage(str(mask_gt_path)))
    dsc = compute_dice_coefficient(mask_gt, mask_pred)
    dscs.append(dsc)

print(f"mean dice coefficient: {np.nanmean(dscs)}:.4f")
print(f"median dice coefficient: {np.nanmedian(dscs):.4f}")
print(f"std dice coefficient: {np.nanstd(dscs):.4f}")
print(np.round(dscs, 4))

mean dice coefficient: 0.6888410730416243:.4f
median dice coefficient: 0.7344
std dice coefficient: 0.1830
[0.8202 0.8515 0.4069 0.6486 0.5084 0.8975]


In [46]:
task_name = "MR_Crohns_Proxy_coronal"
task_id = 3
npz_imgs_folder = all_data_folder / task_name / "data"
npz_weak_masks_folder = all_data_folder / task_name / "masks"
output_dataset_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}"
output_dataset_folder.mkdir(exist_ok=True)

# Train set
images_folder = output_dataset_folder / "imagesTr"
labels_folder = output_dataset_folder / "labelsTr"
images_folder.mkdir(exist_ok=True)
labels_folder.mkdir(exist_ok=True)

npz_imgs_paths = sorted(npz_imgs_folder.glob("*.npz"))
npz_weak_masks_paths = sorted(npz_weak_masks_folder.glob("*.npz"))

for i, (img_file, gt_file) in tqdm(
    enumerate(zip(npz_imgs_paths, npz_weak_masks_paths)),
    total=len(npz_imgs_paths),
):
    img_data = np.load(img_file)
    image_volume = img_data["imgs"]
    gt_data = np.load(gt_file)
    weak_mask_volume = gt_data["gts"]

    # convert the 3-channel image to a single channel image
    image_volume = image_volume[:, :, :, 0]

    # Save the image volume
    saved_image = sitk.GetImageFromArray(image_volume)
    saved_image.SetSpacing((1, 1, 1))
    saved_image.SetOrigin((0, 0, 0))
    saved_image.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_image,
        str(images_folder / f"{task_name}_Training_{i+1:03d}_0000.nii.gz"),
    )


    # Save the weak mask volume
    saved_weak_mask = sitk.GetImageFromArray(weak_mask_volume)
    saved_weak_mask.SetSpacing((1, 1, 1))
    saved_weak_mask.SetOrigin((0, 0, 0))
    saved_weak_mask.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_weak_mask,
        str(labels_folder / f"{task_name}_Training_{i+1:03d}.nii.gz"),
    )


(14, 256, 256) (14, 256, 256)
(19, 256, 256) (19, 256, 256)
(16, 256, 256) (16, 256, 256)
(17, 256, 256) (17, 256, 256)
(10, 256, 256) (10, 256, 256)
(18, 256, 256) (18, 256, 256)
(17, 256, 256) (17, 256, 256)
(17, 256, 256) (17, 256, 256)
(15, 256, 256) (15, 256, 256)
(7, 256, 256) (7, 256, 256)
(23, 256, 256) (23, 256, 256)
(11, 256, 256) (11, 256, 256)
(5, 256, 256) (5, 256, 256)
(12, 256, 256) (12, 256, 256)
(18, 256, 256) (18, 256, 256)
(15, 256, 256) (15, 256, 256)
(18, 256, 256) (18, 256, 256)
(13, 256, 256) (13, 256, 256)
(16, 256, 256) (16, 256, 256)
(17, 256, 256) (17, 256, 256)
(6, 256, 256) (6, 256, 256)
(16, 256, 256) (16, 256, 256)
(13, 256, 256) (13, 256, 256)
(9, 256, 256) (9, 256, 256)
(16, 256, 256) (16, 256, 256)
(14, 256, 256) (14, 256, 256)
(19, 256, 256) (19, 256, 256)
(5, 256, 256) (5, 256, 256)
(12, 256, 256) (12, 256, 256)
(10, 256, 256) (10, 256, 256)
(20, 256, 256) (20, 256, 256)
(10, 256, 256) (10, 256, 256)
(16, 256, 256) (16, 256, 256)
(11, 256, 256) (11, 

In [25]:
generate_dataset_json(
    output_folder=str(output_dataset_folder),
    channel_names={
        encoding: modality for modality, encoding in MODALITY_ENCODINGS.items()
    },
    labels={
        "background": 0,
        "terminal ileum": 1,
    },
    num_training_cases=len(npz_imgs_paths),
    file_ending=".nii.gz",
    regions_class_order=(1,),
    license="see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863",
    reference="see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863",
    dataset_release_date="1.0",
)

In [7]:
task_name = "MR_Crohns_coronal"
task_id = 4

In [54]:
npz_data_folder = all_data_folder / task_name / "train"

output_dataset_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}"
output_dataset_folder.mkdir(exist_ok=True)

# Train set
images_folder = output_dataset_folder / "imagesTr"
labels_folder = output_dataset_folder / "labelsTr"
images_folder.mkdir(exist_ok=True)
labels_folder.mkdir(exist_ok=True)

npz_data_paths = sorted(npz_data_folder.glob("*.npz"))

for i, npz_file in tqdm(enumerate(npz_data_paths), total=len(npz_data_paths)):
    npz_data = np.load(npz_file)
    image_volume = npz_data["imgs"]
    gt_volume = npz_data["gts"]

    # convert the 3-channel image to a single channel image
    image_volume = image_volume[:, :, :, 0]

    # Save the image volume
    saved_image = sitk.GetImageFromArray(image_volume)
    saved_image.SetSpacing((1, 1, 1))
    saved_image.SetOrigin((0, 0, 0))
    saved_image.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_image,
        str(images_folder / f"{task_name}_Training_{i+1:03d}_0000.nii.gz"),
    )

    # Save the gt volume
    saved_gt = sitk.GetImageFromArray(gt_volume)
    saved_gt.SetSpacing((1, 1, 1))
    saved_gt.SetOrigin((0, 0, 0))
    saved_gt.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_gt,
        str(labels_folder / f"{task_name}_Training_{i+1:03d}.nii.gz"),
    )


In [55]:
num_training_cases = len(list(Path(images_folder).glob("*.nii.gz")))
generate_dataset_json(
    output_folder=str(output_dataset_folder),
    channel_names={
        encoding: modality for modality, encoding in MODALITY_ENCODINGS.items()
    },
    labels={
        "background": 0,
        "terminal ileum": 1,
    },
    num_training_cases=num_training_cases,
    file_ending=".nii.gz",
    regions_class_order=(1,),
    license="see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863",
    reference="see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863",
    dataset_release_date="1.0",
)

In [8]:
npz_data_folder = all_data_folder / task_name / "val"

output_dataset_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}"
output_dataset_folder.mkdir(exist_ok=True)

# Train set
images_folder = output_dataset_folder / "imagesTs"
labels_folder = output_dataset_folder / "labelsTs"
images_folder.mkdir(exist_ok=True)
labels_folder.mkdir(exist_ok=True)

npz_data_paths = sorted(npz_data_folder.glob("*.npz"))

for i, npz_file in tqdm(enumerate(npz_data_paths), total=len(npz_data_paths)):
    npz_data = np.load(npz_file)
    image_volume = npz_data["imgs"]
    gt_volume = npz_data["gts"]

    # convert the 3-channel image to a single channel image
    if image_volume.shape[-1] == 3:
        image_volume = image_volume[:, :, :, 0]

    # Save the image volume
    saved_image = sitk.GetImageFromArray(image_volume)
    saved_image.SetSpacing((1, 1, 1))
    saved_image.SetOrigin((0, 0, 0))
    saved_image.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_image,
        str(images_folder / f"{task_name}_Testing_{i+1:03d}_0000.nii.gz"),
    )


    # Save the gt volume
    saved_gt = sitk.GetImageFromArray(gt_volume)
    saved_gt.SetSpacing((1, 1, 1))
    saved_gt.SetOrigin((0, 0, 0))
    saved_gt.SetDirection((1, 0, 0, 0, 1, 0, 0, 0, 1))
    sitk.WriteImage(
        saved_gt,
        str(labels_folder / f"{task_name}_Testing_{i+1:03d}.nii.gz"),
    )

100%|██████████| 7/7 [00:00<00:00, 43.25it/s]


In [9]:
input_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}" / "imagesTs"
output_folder = base_path / "inference" / f"Dataset{task_id:03d}_{task_name}"
output_folder.mkdir(exist_ok=True)
print(f"nnUNetv2_predict -d 3 -i {str(input_folder)} -o {str(output_folder)} -f all -tr nnUNetTrainer_200epochs -c 2d -p nnUNetPlans")

nnUNetv2_predict -d 3 -i D:\Frank\Imperial-FYP\software_archive\data\nnUNet_raw\Dataset004_MR_Crohns_coronal\imagesTs -o d:\Frank\Imperial-FYP\software_archive\inference\Dataset004_MR_Crohns_coronal -f all -tr nnUNetTrainer_200epochs -c 2d -p nnUNetPlans


In [10]:
mask_prediction_paths = sorted(output_folder.glob("*.nii.gz"))
mask_gt_paths = sorted((Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}" / "labelsTs").glob("*.nii.gz"))
mask_prediction_paths

[WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset004_MR_Crohns_coronal/MR_Crohns_coronal_Testing_001.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset004_MR_Crohns_coronal/MR_Crohns_coronal_Testing_002.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset004_MR_Crohns_coronal/MR_Crohns_coronal_Testing_003.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset004_MR_Crohns_coronal/MR_Crohns_coronal_Testing_004.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset004_MR_Crohns_coronal/MR_Crohns_coronal_Testing_005.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset004_MR_Crohns_coronal/MR_Crohns_coronal_Testing_006.nii.gz'),
 WindowsPath('d:/Frank/Imperial-FYP/software_archive/inference/Dataset004_MR_Crohns_coronal/MR_Crohns_coronal_Testing_007.nii.gz')]

In [14]:
dscs = []
for mask_pred_path, mask_gt_path in zip(mask_prediction_paths, mask_gt_paths):
    mask_pred = sitk.GetArrayFromImage(sitk.ReadImage(str(mask_pred_path)))
    mask_gt = sitk.GetArrayFromImage(sitk.ReadImage(str(mask_gt_path)))
    dsc = compute_dice_coefficient(mask_gt, mask_pred)
    dscs.append(dsc)

print(f"mean dice coefficient: {np.nanmean(dscs):.4f}")
print(f"median dice coefficient: {np.nanmedian(dscs):.4f}")
print(f"std dice coefficient: {np.nanstd(dscs):.4f}")
print(np.round(dscs, 4))

mean dice coefficient: 0.2613
median dice coefficient: 0.1663
std dice coefficient: 0.1836
[0.6004 0.0542 0.1663 0.1131 0.314  0.1443 0.4364]


In [17]:
def show_mask(mask, ax):
    color = np.array([251 / 255, 252 / 255, 30 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [30]:
from matplotlib import pyplot as plt
import numpy as np


all_data_folder = base_path / "lab" / "Npz_files"
task_name = "MR_Crohns_axial"
task_id = 2
input_folder = Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}" / "imagesTs"
output_folder = base_path / "inference" / f"Dataset{task_id:03d}_{task_name}"
mask_prediction_paths = sorted(output_folder.glob("*.nii.gz"))
mask_gt_paths = sorted((Path(nnUNet_raw) / f"Dataset{task_id:03d}_{task_name}" / "labelsTs").glob("*.nii.gz"))
image_paths = sorted(input_folder.glob("*.nii.gz"))
dscs = []
np.random.seed(42)
for i, (image_path, mask_pred_path, mask_gt_path) in enumerate(zip(image_paths, mask_prediction_paths, mask_gt_paths)):
    image = sitk.GetArrayFromImage(sitk.ReadImage(str(image_path)))
    mask_pred = sitk.GetArrayFromImage(sitk.ReadImage(str(mask_pred_path)))
    mask_gt = sitk.GetArrayFromImage(sitk.ReadImage(str(mask_gt_path)))
    dsc = compute_dice_coefficient(mask_gt, mask_pred)
    dscs.append(dsc)
    # show ground truth and prediction on the same image
    # fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    # img_idx = np.random.randint(0, image.shape[0])
    # ax[0].imshow(image[img_idx], cmap="gray")
    # show_mask(mask_gt[img_idx], ax[0])
    # ax[0].set_title("Ground truth", fontsize=16, fontweight="bold")
    # ax[1].imshow(image[img_idx], cmap="gray")
    # show_mask(mask_pred[img_idx], ax[1])
    # ax[1].set_title(f"Prediction, DSC: {dsc:.4f}", fontsize=16, fontweight="bold")
    # ax[0].axis("off")
    # ax[1].axis("off")
    # plt.tight_layout()
    # fig.savefig(f"results_{img_idx}.png", dpi=300)
    # plt.show()
    img_idx = np.random.randint(0, image.shape[0])
    print(f"Image {i+1} DSC: {dsc:.4f}, img_idx: {img_idx}")
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(image[img_idx], cmap="gray")
    show_mask(mask_pred[img_idx], ax)
    ax.axis("off")
    fig.savefig(f"results_{i+1}_medsam.png", dpi=300)
    plt.close(fig)

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(image[img_idx], cmap="gray")
    show_mask(mask_gt[img_idx], ax)
    ax.axis("off")
    fig.savefig(f"results_{i+1}_gt.png", dpi=300)
    plt.close(fig)



print(f"Average DSC: {np.nanmean(dscs):.4f}")
print(f"Median DSC: {np.nanmedian(dscs):.4f}")
print(f"Std DSC: {np.nanstd(dscs):.4f}")
print(f"Max DSC: {np.nanmax(dscs):.4f}")

Image 1 DSC: 0.8202, img_idx: 6
Image 2 DSC: 0.8515, img_idx: 14
Image 3 DSC: 0.4069, img_idx: 10
Image 4 DSC: 0.6486, img_idx: 7
Image 5 DSC: 0.5084, img_idx: 20
Image 6 DSC: 0.8975, img_idx: 6
Average DSC: 0.6888
Median DSC: 0.7344
Std DSC: 0.1830
Max DSC: 0.8975


In [22]:
def get_bbox_from_mask(mask):
    """Returns a bounding box from a mask"""
    y_indices, x_indices = np.where(mask > 0)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    # add perturbation to bounding box coordinates
    H, W = mask.shape
    x_min = max(0, x_min - np.random.randint(0, 20))
    x_max = min(W, x_max + np.random.randint(0, 20))
    y_min = max(0, y_min - np.random.randint(0, 20))
    y_max = min(H, y_max + np.random.randint(0, 20))

    return np.array([x_min, y_min, x_max, y_max])

In [32]:
# SAM 
import torch
from segment_anything import sam_model_registry, SamPredictor
from segment_anything.utils.transforms import ResizeLongestSide

model = sam_model_registry["vit_b"](
    checkpoint="../work_dir/SAM/sam_vit_b_01ec64.pth",
).to('cuda:0')

model_predictor = SamPredictor(model)
sam_trans = ResizeLongestSide(model.image_encoder.img_size)

model.eval()
npz_ts_folder = all_data_folder / task_name / "val"
npz_ts_paths = sorted(npz_ts_folder.glob("*.npz"))
dscs = []
np.random.seed(42)
idx = [6, 14, 10, 7, 20, 6]
for i, npz_ts_path in enumerate(npz_ts_paths):
    npz = np.load(npz_ts_path)
    imgs = npz["imgs"]
    masks = npz["gts"]
    segs = []
    for img, mask in zip(imgs, masks):
        bbox = get_bbox_from_mask(mask)
        model_predictor.set_image(img)
        sam_seg, _, _ = model_predictor.predict(
            point_coords=None, box=bbox, multimask_output=False
        )

        segs.append(sam_seg[0])
    segs = np.stack(segs, axis=0)
    dsc = compute_dice_coefficient(masks > 0, segs > 0)
    dscs.append(dsc)
    img_idx = idx[i]
    # # show ground truth and prediction on the same image
    # fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    # ax[0].imshow(imgs[img_idx], cmap="gray")
    # show_mask(masks[img_idx], ax[0])
    # ax[0].set_title("Ground truth", fontsize=16, fontweight="bold")
    # ax[1].imshow(imgs[img_idx], cmap="gray")
    # show_mask(segs[img_idx], ax[1])
    # ax[1].set_title(f"Prediction, DSC: {dsc:.4f}", fontsize=16, fontweight="bold")
    # ax[0].axis("off")
    # ax[1].axis("off")
    # plt.tight_layout()
    # plt.show()
    print(f"Image {i+1} DSC: {dsc:.4f}, img_idx: {img_idx}")
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(imgs[img_idx], cmap="gray")
    show_mask(segs[img_idx], ax)
    ax.axis("off")
    fig.savefig(f"results_{i+1}_baseline.png", dpi=300)
    plt.close(fig)


print(f"Average DSC: {np.nanmean(dscs):.4f}")
print(f"Median DSC: {np.nanmedian(dscs):.4f}")
print(f"Std DSC: {np.nanstd(dscs):.4f}")
print(f"Max DSC: {np.nanmax(dscs):.4f}")


Image 1 DSC: 0.6987, img_idx: 6
Image 2 DSC: 0.6845, img_idx: 14
Image 3 DSC: 0.5315, img_idx: 10
Image 4 DSC: 0.5331, img_idx: 7
Image 5 DSC: 0.5821, img_idx: 20
Image 6 DSC: 0.6758, img_idx: 6
Average DSC: 0.6176
Median DSC: 0.6290
Std DSC: 0.0710
Max DSC: 0.6987


In [39]:
import os
weak_mask_showcases = []
for f in os.listdir('baseline_weakmask_showcase'):
    if f.endswith('.png'):
        if 'dsc' not in f and 'axial' in f:
            if f.startswith('A') or f.startswith('I'):
                identifier = f[:-7]
                weak_mask_showcases.append(identifier)

In [40]:
weak_mask_showcases

['A005_axial', 'A006_axial', 'A008_axial', 'A012_axial', 'A015_axial']

In [41]:
original_crohns = "../processed_crohns/Crohns2023Axial/"
os.mkdir('weakmask_test')
for f in weak_mask_showcases:
    shutil.copy(original_crohns + f + '.nii.gz', 'weakmask_test')

In [42]:
import xml.etree.ElementTree as ET


def get_centerline_points(centerline_path):
    crohns_centerline = ET.parse(centerline_path)
    root = crohns_centerline.getroot()
    centerline_points = []
    for path in root:
        if 'name' not in path.attrib:
            continue

        for point in path:
            centerline_points.append([int(point.attrib['x']), int(point.attrib['y']), int(point.attrib['z'])])
    
    crohns_centerline_size = int(len(centerline_points) * 0.2)
    return np.array(centerline_points[:crohns_centerline_size])

In [43]:
def get_bbox_from_centerline(centerline_points):
    min_x = np.min(centerline_points[:, 0])
    max_x = np.max(centerline_points[:, 0])
    min_y = np.min(centerline_points[:, 1])
    max_y = np.max(centerline_points[:, 1])
    min_z = np.min(centerline_points[:, 2])
    max_z = np.max(centerline_points[:, 2])
    
    # the top left corner of the bounding box
    index = (int(min_x), int(min_y), int(min_z))
    # the size of the bounding box
    size = (int(max_x - min_x + 1), int(max_y - min_y + 1), int(max_z - min_z + 1))
    return index, size

In [50]:
def get_bbox_from_mask(mask):
    """Returns a bounding box from a mask"""
    y_indices, x_indices = np.where(mask > 0)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    # add perturbation to bounding box coordinates
    H, W = mask.shape
    x_min = max(0, x_min - np.random.randint(0, 20))
    x_max = min(W, x_max + np.random.randint(0, 20))
    y_min = max(0, y_min - np.random.randint(0, 20))
    y_max = min(H, y_max + np.random.randint(0, 20))

    return np.array([x_min, y_min, x_max, y_max])

In [51]:
def compute_dice_coefficient(mask_gt, mask_pred):
    """Compute soerensen-dice coefficient.

    compute the soerensen-dice coefficient between the ground truth mask `mask_gt`
    and the predicted mask `mask_pred`.

    Args:
      mask_gt: 3-dim Numpy array of type bool. The ground truth mask.
      mask_pred: 3-dim Numpy array of type bool. The predicted mask.

    Returns:
      the dice coeffcient as float. If both masks are empty, the result is NaN
    """
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN
    volume_intersect = (mask_gt & mask_pred).sum()
    return 2 * volume_intersect / volume_sum

In [61]:
from segment_anything import sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide


original_crohns_centerline = "../processed_crohns/Crohns2023AxialCenterlines/A015_axial.xml"
image = sitk.ReadImage('weakmask_test/A015_axial.nii.gz')
image = sitk.GetArrayFromImage(image)
# label = sitk.ReadImage("../processed_crohns/Crohns2023AxialLabels/A015_axial.nii.gz")
# label = sitk.GetArrayFromImage(label)
# print(np.unique(label))
label = np.zeros_like(image)

centerline_points = get_centerline_points(original_crohns_centerline)
index, size = get_bbox_from_centerline(centerline_points)

xmin, ymin, zmin = index
xmax, ymax, zmax = np.array(index) + np.array(size)

label[zmin:zmax, ymin:ymax, xmin:xmax] = 1

medsam_model = sam_model_registry['vit_b'](
    checkpoint='../work_dir/MR_Crohns_axial/sam_model_best.pth',
).to('cuda')

sam_trans = ResizeLongestSide(medsam_model.image_encoder.img_size)

medsam_segs = []
for img, gt in zip(image, label):
    bbox = get_bbox_from_mask(gt)
    H, W = img.shape[:2]
    resize_img = sam_trans.apply_image(img)
    resize_img_tensor = torch.as_tensor(resize_img.transpose(2, 0, 1)).to(
        'cuda'
    )
    input_image = medsam_model.preprocess(resize_img_tensor[None, :, :, :])
    with torch.no_grad():
        image_embedding = medsam_model.image_encoder(input_image.to('cuda'))
        # convert box to 1024x1024 grid
        bbox = sam_trans.apply_boxes(bbox, (H, W))
        box_torch = torch.as_tensor(bbox, dtype=torch.float, device='cuda')
        if len(box_torch.shape) == 2:
            box_torch = box_torch[:, None, :]
        sparse_embeddings, dense_embeddings = medsam_model.prompt_encoder(
            points=None,
            boxes=box_torch,
            masks=None,
        )
        medsam_seg_prob, _ = medsam_model.mask_decoder(
            image_embeddings=image_embedding.to('cuda'),
            image_pe=medsam_model.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False,
        )
        medsam_seg_prob = torch.sigmoid(medsam_seg_prob)
        # convert soft mask to hard mask
        medsam_seg_prob = medsam_seg_prob.cpu().numpy().squeeze()
        medsam_seg = (medsam_seg_prob > 0.5).astype(np.uint8)
        medsam_segs.append(medsam_seg)

medsam_segs = np.stack(medsam_segs, axis=0)
medsam_dsc = compute_dice_coefficient(label > 0, medsam_segs > 0)

print(medsam_dsc)
    

ValueError: zero-size array to reduction operation minimum which has no identity